**Assign the best possible scheduling to match participants with their preferences**

In [1]:
# list of participants and their choices
participants = {'Jonathan':[4, 1, 2], 'Nishan':[1, 2, 3], 'Terlo':[1, 3, 2], 'Michael':[3, 2, 1]}

In [2]:
from itertools import chain

# generate the list of unique preferences possible
preferences = list(set(chain(*[*participants.values()])))

In [3]:
from itertools import cycle

def repeat_list(add_list, repeats, total):
    new_list = []
    stack = cycle(add_list)
    for item in stack:
        new_list.extend([item]*repeats)
        if len(new_list) == total:
            break
    return new_list

In [4]:
import pandas as pd

# total combinations
total = 3**len(preferences)
repeats = int(total/3) # initial condition

# create exhaustive list of all choices
all_choices = pd.DataFrame()

for index, value in zip(participants.keys(), participants.values()):
    all_choices[index] = repeat_list(value, repeats, total)
    repeats = int(repeats/3)

In [5]:
# drop all trivial choices
trivial_indices = []

for index, rows in all_choices.iterrows():
    if len(set(list(rows))) != len(rows):
        trivial_indices.append(index)

non_trivial_choices = all_choices.drop(trivial_indices).reset_index(drop=True)

**A lower score is a better score.**

In [6]:
def highlight(df, threshold, column):
    maximum = pd.Series(data=False, index=df.index)
    maximum[column] = df.loc[column] == threshold
    return ['background-color: blue' if maximum.any() else '' for value in maximum]

In [7]:
# scoring function
scores = []
for index, rows in non_trivial_choices.iterrows():
    score = 0
    participant = iter(participants)
    for row_index in rows:
        next_participant = next(participant)
        score += participants[next_participant].index(row_index)
    scores.append(score)
non_trivial_choices['Score'] = scores
non_trivial_choices = non_trivial_choices.sort_values(by='Score', ascending=True)
non_trivial_choices.style.apply(highlight, threshold=non_trivial_choices['Score'].min(), column='Score', axis=1)

,Jonathan,Nishan,Terlo,Michael,Score
2,4,2,1,3,1
0,4,1,3,2,2
1,4,1,2,3,2
4,4,3,1,2,3
3,4,2,3,1,4
5,4,3,2,1,6


In [8]:
best = non_trivial_choices.loc[non_trivial_choices['Score'] == non_trivial_choices['Score'].min()]
best = best.drop(["Score"], axis=1)
best.sample() # random choice in case of a tie

,Jonathan,Nishan,Terlo,Michael
2,4,2,1,3
